In [1]:
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import os
import sys
import pandas as pd, numpy as np
import timeit
import joblib
from copy import deepcopy
import pandas as pd
import time

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.python.client import device_lib
from sklearn.metrics import accuracy_score, roc_auc_score

import pathlib
sys.path.insert(0, os.getcwd().split('scripts')[0])

print(sys.version, sys.platform, sys.executable) #Displays what environment you are actually using.

from src import models
from src import sparse_soft_trees
from src import utils
from data import data_utils


from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def get_available_cpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'CPU']


os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

2025-03-01 15:52:02.841138: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0] linux /home/gridsan/shibal/.conda/envs/MOETF29/bin/python


# Load dataset and standardize

In [3]:
data = load_breast_cancer()
x = data.data
y = data.target

seed = 8
x_train_valid, x_test, y_train_valid, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=seed)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, stratify=y_train_valid, random_state=seed)

assert len(y.shape)==1
# print("Class-set (Train)", np.unique(data_processed.y_train_processed))
# print("Class-set (Valid)", np.unique(data_processed.y_valid_processed))
# print("Class-set (Train-Valid)", np.unique(data_processed.y_train_valid_processed))
# print("Class-set (Test)", np.unique(data_processed.y_test_processed))
num_classes = y.max(axis=0)+1
print("====================num_classes:", num_classes)
num_features = x.shape[1]
print("====================num_features:", num_features)

x_preprocessor = StandardScaler()
x_train_processed = x_preprocessor.fit_transform(x_train)
x_valid_processed = x_preprocessor.transform(x_valid)
x_test_processed = x_preprocessor.transform(x_test)

y_train_processed = y_train
y_valid_processed = y_valid
y_test_processed = y_test


====================num_classes: 2
====================num_features: 30


# Note 

Note that there are two configurations:
1. Fixed Group-L0 regularization:
use_annealing=False, kernel_constraint=varied across a range of params, number of epochs influence the degree of sparsity.
2. Anneals the Group-L0 regularization:
use_annealing=True, kernel_constraint=100, temperature = 0.01, epochs influence the degree of sparsity in the model

Below we show running the model with a particular choice of hyperparameters for case (2). 

Since we solve the unconstrained feature selection problem, we need to sweep these parameters to find settings where the model the satisfies a desired budget of features. See main_classification_public_data.py for hyperparameter tuning with optuna and SkinnyTrees-Results.ipynb can be used to compile results.

# Define and fit the model

In [4]:
constant_batch_size = 16
batch_size_scaler = 1
batch_size = constant_batch_size*batch_size_scaler
constant_learning_rate = 0.1
early_stopping = False


num_train_samples = x_train.shape[0]
epochs = 1000

if num_train_samples % batch_size == 0:
    epoch_step = num_train_samples / batch_size
else:
    epoch_step = int(num_train_samples / batch_size) + 1

#         epochs = trial.suggest_int('epochs_new', 5, epochs)
print("==============No LR scheduler, Epochs:", epochs, "Batch-size:", batch_size)
print("==============epochs:", epochs)
learning_rate = constant_learning_rate
lr_schedule = utils.ConstantLearningRate(
    learning_rate
)
optim = tf.keras.optimizers.SGD(lr_schedule)

### Soft Decision Tree parameters 
num_trees = 20
depth = 4

activation = tf.keras.activations.sigmoid

use_annealing = True
kernel_constraint = 100
temperature = 0.01
kernel_l2 = 0.1
kernel_l2 = kernel_l2/(num_trees*(2**depth - 1))
kernel_regularizer = tf.keras.regularizers.L2(kernel_l2)
if use_annealing:
    kernel_constraint = kernel_constraint/num_features
    temperature = temperature
    kernel_constraint=sparse_soft_trees.ProximalGroupL0(lr=lr_schedule, lam=kernel_constraint, temperature=temperature, use_annealing=True, name='ProximalGroupL0')
else:
    kernel_constraint = kernel_constraint/num_features
    kernel_constraint=sparse_soft_trees.ProximalGroupL0(lr=lr_schedule, lam=kernel_constraint, use_annealing=False, name='ProximalGroupL0')
print("===========kernel_regularizer:", kernel_regularizer)
print("===========kernel_constraint:", kernel_constraint)


### Optimization parameters
leaf_dims = (num_classes, )
x = tf.keras.layers.Input(name='input', shape=x_train_processed.shape[1:])
submodel = models.create_model(
    x,
    num_trees,
    depth,
    leaf_dims,
    activation=activation,
    kernel_regularizer=kernel_regularizer,
    kernel_constraint=kernel_constraint,
)
x = submodel.input
outputs = submodel(x)
# print(outputs)
ypred = tf.keras.layers.Activation('linear')(outputs)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model = tf.keras.Model(inputs=x, outputs=ypred)

model.summary()

monitor = 'val_accuracy'
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optim, metrics=metrics)
cb = sparse_soft_trees.SparsityHistory()
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    cb
]    
if early_stopping:
    callbacks.append(
        sparse_soft_trees.EarlyStopping(
            config['cardinality'], monitor=monitor, patience=100, verbose=1, mode='auto', restore_best_weights=True
        ),
    )
# print("====================y.shape", data_processed.y_train_processed.shape)
if len(get_available_gpus())==0:
    history = model.fit(x=x_train_processed, 
              y=y_train_processed,
              epochs=epochs, 
              batch_size=batch_size, 
              shuffle=True,
              callbacks=callbacks,
              validation_data=(x_valid_processed, y_valid_processed),
              verbose=1, 
              )  
else:
    with tf.device(get_available_gpus()[0]):
        history = model.fit(x=x_train_processed, 
                  y=y_train_processed,
                  epochs=epochs, 
                  batch_size=batch_size, 
                  shuffle=True,
                  callbacks=callbacks,
                  validation_data=(x_valid_processed, y_valid_processed),
                  verbose=0, 
                  )  

==============No LR scheduler, Epochs: 1000 Batch-size: 16
==============epochs: 1000
===========kernel_regularizer: <keras.regularizers.L2 object at 0x7f2933e8d5d0>
===========kernel_constraint: <src.sparse_soft_trees.ProximalGroupL0 object at 0x7f2933e8d360>


2025-03-01 15:52:06.249593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/gridsan/shibal/.conda/envs/MOETF29/lib/:/home/gridsan/shibal/.conda/envs/MOETF29/lib/python3.10/site-packages/nvidia/cudnn/lib
2025-03-01 15:52:06.249630: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-01 15:52:06.249651: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d-5-3-4): /proc/driver/nvidia/version does not exist
2025-03-01 15:52:06.250079: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations,

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 30)]              0         
                                                                 
 model (Functional)          (None, 2)                 9940      
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                                 
Total params: 9,940
Trainable params: 9,940
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
23/23 [==============================] - 4s 28ms/step - loss: 0.4700 - accuracy: 0.8489 - lam: 0.3406 - nnz: 30.0000 - group-l0-reg: 10.2194 - val_loss: 0.3298 - val_accuracy: 0.9560 - val_lam: 0.6849 - val_nnz: 30.0000 - val_group-l0-reg: 20.5466
Epoch 2/1000
23/23 [==============================] - 0s 7ms

23/23 [==============================] - 0s 7ms/step - loss: 0.0942 - accuracy: 0.9863 - lam: 3.3286 - nnz: 30.0000 - group-l0-reg: 99.8567 - val_loss: 0.1240 - val_accuracy: 0.9670 - val_lam: 3.3291 - val_nnz: 30.0000 - val_group-l0-reg: 99.8732
Epoch 30/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0933 - accuracy: 0.9863 - lam: 3.3295 - nnz: 30.0000 - group-l0-reg: 99.8861 - val_loss: 0.1235 - val_accuracy: 0.9670 - val_lam: 3.3300 - val_nnz: 30.0000 - val_group-l0-reg: 99.8992
Epoch 31/1000
23/23 [==============================] - 0s 8ms/step - loss: 0.0923 - accuracy: 0.9863 - lam: 3.3303 - nnz: 30.0000 - group-l0-reg: 99.9095 - val_loss: 0.1231 - val_accuracy: 0.9670 - val_lam: 3.3307 - val_nnz: 30.0000 - val_group-l0-reg: 99.9199
Epoch 32/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0913 - accuracy: 0.9863 - lam: 3.3309 - nnz: 30.0000 - group-l0-reg: 99.9281 - val_loss: 0.1228 - val_accuracy: 0.9670 - val_lam: 3.3312 - val_nnz: 30.000

23/23 [==============================] - 0s 7ms/step - loss: 0.0767 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 99.9999 - val_loss: 0.1183 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 99.9999
Epoch 61/1000
23/23 [==============================] - 0s 8ms/step - loss: 0.0764 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 99.9999 - val_loss: 0.1182 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 99.9999
Epoch 62/1000
23/23 [==============================] - 0s 5ms/step - loss: 0.0760 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 99.9999 - val_loss: 0.1181 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 99.9999
Epoch 63/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0758 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1180 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.00

...

23/23 [==============================] - 0s 4ms/step - loss: 0.0279 - accuracy: 0.9973 - lam: 3.3333 - nnz: 25.0000 - group-l0-reg: 83.3333 - val_loss: 0.1272 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 25.0000 - val_group-l0-reg: 83.3333
Epoch 867/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0279 - accuracy: 0.9973 - lam: 3.3333 - nnz: 25.0000 - group-l0-reg: 83.3333 - val_loss: 0.1272 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 25.0000 - val_group-l0-reg: 83.3333
Epoch 868/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0279 - accuracy: 0.9973 - lam: 3.3333 - nnz: 25.0000 - group-l0-reg: 83.3333 - val_loss: 0.1272 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 25.0000 - val_group-l0-reg: 83.3333
Epoch 869/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0279 - accuracy: 0.9973 - lam: 3.3333 - nnz: 25.0000 - group-l0-reg: 83.3333 - val_loss: 0.1272 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 25.

23/23 [==============================] - 0s 5ms/step - loss: 0.0284 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1265 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 898/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0282 - accuracy: 1.0000 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1264 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 899/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0281 - accuracy: 1.0000 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1263 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 900/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0281 - accuracy: 1.0000 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1262 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.

23/23 [==============================] - 0s 5ms/step - loss: 0.0273 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1245 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 929/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0272 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1246 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 930/1000
23/23 [==============================] - 0s 5ms/step - loss: 0.0272 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1245 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 931/1000
23/23 [==============================] - 0s 5ms/step - loss: 0.0272 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1245 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.

23/23 [==============================] - 0s 5ms/step - loss: 0.0267 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1250 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 960/1000
23/23 [==============================] - 0s 5ms/step - loss: 0.0268 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1253 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 961/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0267 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1253 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 962/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0267 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1252 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.

23/23 [==============================] - 0s 5ms/step - loss: 0.0263 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1259 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 991/1000
23/23 [==============================] - 0s 5ms/step - loss: 0.0263 - accuracy: 1.0000 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1260 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 992/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0263 - accuracy: 1.0000 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1260 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.0000 - val_group-l0-reg: 80.0000
Epoch 993/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.0263 - accuracy: 0.9973 - lam: 3.3333 - nnz: 24.0000 - group-l0-reg: 80.0000 - val_loss: 0.1258 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 24.

# Evaluate the model

In [7]:
number_of_epochs_it_ran = len(history.history['loss'])

val_loss_history = history.history['val_loss']

if early_stopping:
    best_epoch = callbacks[-1].best_epoch # np.argmin(val_loss_history) + 1
else:
    best_epoch = len(val_loss_history)
print("============best_epoch:", best_epoch)
print("============number_of_epochs_it_ran:", number_of_epochs_it_ran)

feature_sparsity_history = cb.selected_features[1:(best_epoch+1)]
approximate_feature_sparsity_history = cb.approximately_selected_features[1:(best_epoch+1)]

with tf.device(get_available_cpus()[0]):
    # Check for infinite loss
    training_loss = model.evaluate(x_train_processed,
                                   y_train_processed,
                                   batch_size=batch_size,
                                   verbose=0)

    if np.isfinite(np.sum(training_loss)) or ~np.isnan(np.sum(training_loss)):
        # Evaluation

        y_valid_pred = model.predict(x_valid_processed)
        y_valid_prob = tf.nn.softmax(y_valid_pred, axis=1).numpy()
        y_valid_pred_classes = np.argmax(y_valid_prob, axis=1)
        classes = np.shape(y_valid_prob)[1]
        if classes==2:
            y_valid_prob = tf.gather(y_valid_prob, indices=[1], axis=1).numpy()
        accuracy_valid = accuracy_score(y_valid_processed, y_valid_pred_classes)
        auc_valid = roc_auc_score(
            y_valid_processed,
            y_valid_prob,
            multi_class='ovo'
        )

        y_test_pred = model.predict(x_test_processed)
        y_test_prob = tf.nn.softmax(y_test_pred, axis=1).numpy()
        y_test_pred_classes = np.argmax(y_test_prob, axis=1)
        if classes==2:
            y_test_prob = tf.gather(y_test_prob, indices=[1], axis=1).numpy()
        accuracy_test = accuracy_score(y_test_processed, y_test_pred_classes)
        auc_test = roc_auc_score(
            y_test_processed,
            y_test_prob,
            multi_class='ovo'
        )
        print('accuracy (valid):', accuracy_valid)
        print('accuracy (test):', accuracy_test)
        print('auc (valid):', auc_valid)
        print('auc (test):', auc_test)
print("num_epochs", number_of_epochs_it_ran)
# print("val_loss_history", val_loss_history)
# print("feature_sparsity_history", feature_sparsity_history)    
print("feature_sparsity (number of nonzeros)", feature_sparsity_history[-1])    


============best_epoch: 1000
============number_of_epochs_it_ran: 1000
4/4 [==============================] - 0s 2ms/step
accuracy (valid): 0.967032967032967
accuracy (test): 0.9824561403508771
auc (valid): 0.9901960784313726
auc (test): 0.9993386243386243
num_epochs 1000
feature_sparsity (number of nonzeros) 24
